In [1]:
import catboost

In [2]:
import pandas as pd

In [3]:
train_set = pd.read_csv('train.csv', sep='\t')

In [4]:
train_set[:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [5]:
train_set.shape

(15000000, 19)

In [9]:
train_set['b2c_c2c'].value_counts()

B2C    10634413
C2C     4365587
Name: b2c_c2c, dtype: int64

In [11]:
train_set['shipment_method_id'].value_counts()

0     9341444
1     2955149
2      856653
3      780997
4      297472
5      253508
6      176007
7      137404
8      125093
9       39639
10      27081
11       5906
13       1545
12       1077
14        447
15        431
16         48
19         21
17         20
18         17
21         16
20         13
22          6
24          4
26          2
Name: shipment_method_id, dtype: int64

In [13]:
train_set['shipping_fee'].value_counts()

0.00      9000100
3.50       327017
3.75       223857
4.00       186832
3.00       186692
           ...   
47.08           1
89.90           1
99.85           1
136.85          1
74.57           1
Name: shipping_fee, Length: 7044, dtype: int64

In [14]:
train_set['carrier_min_estimate'].value_counts()

 3    9341496
 2    5649003
 1       7451
-1       1095
 6        524
 0        431
Name: carrier_min_estimate, dtype: int64

In [15]:
train_set['carrier_max_estimate'].value_counts()

 5     13055377
 9      1154125
 8       780997
 1         7882
-1         1095
 25         524
Name: carrier_max_estimate, dtype: int64

In [16]:
train_set['weight_units'].value_counts()

1    14999964
2          36
Name: weight_units, dtype: int64

In [17]:
train_set['weight'].value_counts()

0        4792627
32       1229838
4        1183041
3        1140370
16        853087
          ...   
707            1
980            1
10960          1
5920           1
505            1
Name: weight, Length: 1298, dtype: int64

In [18]:
def add_func(row):
    acct = row['acceptance_scan_timestamp']
    dd = row['deliver_date']
    cdate = datetime.strptime(acct.split()[0], "%Y-%m-%d")
    ddate = datetime.strptime(dd, "%Y-%m-%d")
    delta = cdate - ddate
    return delta.days

In [ ]:
train_set['deliver_day'] = train_set.apply(add_func, axis=1)

In [6]:
quiz_set = pd.read_csv('quiz.tsv', sep='\t')

In [8]:
quiz_set[:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,260031,1.0,2019-11-15 05:11:00.000-07:00,0,0.0,3,5,84116,34788,2,28.00,1,2019-11-13 19:28:44.000-07:00,NaN,4,1,PACKAGE_THICK_ENVELOPE,15000001
1,B2C,4854,1.0,2019-11-28 02:03:00.000-05:00,0,0.0,3,5,04901,54904,9,9.95,1,2019-11-26 14:36:21.000-05:00,NaN,13,1,PACKAGE_THICK_ENVELOPE,15000002
2,B2C,1987,1.0,2019-12-03 00:02:00.000-08:00,0,0.0,3,5,90015,80918,2,16.25,1,2019-12-02 07:39:41.000-08:00,NaN,6,1,PACKAGE_THICK_ENVELOPE,15000003
3,B2C,43799,1.0,2019-12-14 19:56:00.000-05:00,0,0.0,3,5,04007,49036,16,13.65,1,2019-12-12 14:57:31.000-05:00,NaN,4,1,PACKAGE_THICK_ENVELOPE,15000004
4,B2C,3660,0.0,2019-12-19 14:30:00.000-08:00,0,0.0,3,5,95841,74074,1,13.99,1,2019-12-18 16:26:43.000-08:00,NaN,2,1,LARGE_ENVELOPE,15000005


In [10]:
quiz_set['b2c_c2c'].value_counts()

B2C    1722898
C2C     777102
Name: b2c_c2c, dtype: int64

In [12]:
quiz_set['shipment_method_id'].value_counts()

0     1533017
1      520278
3      156470
2      133287
4       40281
5       39942
6       25976
7       21315
8       17684
9        6521
10       3734
11       1144
13        244
15         55
12         32
14         16
16          2
18          1
17          1
Name: shipment_method_id, dtype: int64

In [14]:
zip_code = set(train_set['item_zip']) | set(train_set['buyer_zip']) | set(quiz_set['item_zip']) | set(quiz_set['buyer_zip'])

In [15]:
len(zip_code)

78535

In [16]:
import json

In [17]:
json.dump(list(zip_code), open('zip_code_list.json', 'w', encoding='utf-8'))

In [22]:
train_set[train_set['item_zip'].isnull()]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
5914552,C2C,5971014,NaN,2018-12-15 15:43:05.000-08:00,2,0.0,2,9,NaN,80219-3401,8,302.3,1,2018-12-09 16:53:56.000-08:00,2018-12-18,0,1,PACKAGE_THICK_ENVELOPE,5914553


In [23]:
train_set[train_set['buyer_zip'].isnull()]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
4845032,C2C,1343135,NaN,2018-07-30 11:34:00.000-05:00,12,0.0,-1,-1,65804,NaN,4,5.5,1,2018-07-28 17:03:33.000-05:00,2018-08-04,5,1,PACKAGE_THICK_ENVELOPE,4845033


In [20]:
quiz_set['item_zip'].isnull().sum()

0

In [21]:
quiz_set['item_zip'].isnull().sum()

0